In [ ]:
# PMOD A: 1:RX, 2:TX. baud 9600

In [26]:
from time import sleep, time

RX_FIFO = 0x00
TX_FIFO = 0x04
#Status Reg
STAT_REG = 0x08

RX_VALID = 0
RX_FULL = 1
TX_EMPTY = 2
TX_FULL = 3
IS_INTR = 4
OVERRUN_ERR = 5
FRAME_ERR = 6
PARITY_ERR =7

#Ctrl Reg
CTRL_REG = 0x0C

RST_TX = 0
RST_RX = 1
INTR_EN = 4

def get_bit(num,pos):
    return (num&1<<pos)>>pos

def setup_ctrl_reg(uart):
    # Reset FIFOs, disable interrupts
    uart.write(CTRL_REG, 1<<RST_TX | 1<<RST_RX)
    sleep(1)
    uart.write(CTRL_REG,0)
    sleep(1)

def current_status(uart):
    """Returns object that specifies current status of axi core"""
    status = uart.read(STAT_REG)
    return {'RX_VALID':get_bit(status,RX_VALID),
           'RX_FULL':get_bit(status, RX_FULL),
           'TX_EMPTY':get_bit(status, TX_EMPTY),
           'TX_FULL':get_bit(status, TX_FULL),
           'IS_INTR':get_bit(status, IS_INTR),
           'OVERRUN_ERR':get_bit(status, OVERRUN_ERR),
           'FRAME_ERR':get_bit(status, FRAME_ERR),
           'PARITY_ERR':get_bit(status, PARITY_ERR)}

def read_uart(uart, count, timeout = 10):
    # status = current_status(uart) bad idea
    buf = ""
    stop_time = time() + timeout
    for i in range(count):
        # Wait till RX fifo has valid data, stop waiting if timeoutpasses
        while (not (uart.read(STAT_REG) & 1<<RX_VALID)) and (time()<stop_time):
            sleep(0.1)
        if time()>stop_time:
            break
        buf += chr(uart.read(RX_FIFO))
    return buf

def write_uart(uart, buf, timeout = 10):
    """
    buf: iterable
    
    """
    stop_time = time() + timeout
    wr_count = 0
    for i in buf:
        #Wait while TX FIFO is Full, stop waiting if timeout passes 
        while (uart.read(STAT_REG) & 1<<TX_FULL) and (time()<stop_time):
            sleep(0.1)
        # Check timeout
        if time()>stop_time:
            break
        uart.write(TX_FIFO, ord(i))
        wr_count += 1
    return wr_count   

def readline_uart(uart):
    buf = read_uart(uart, 1)
    if len(buf) ==0:
        return ""
    while '\n' not in buf:
        buf += read_uart(uart, 1)
    return buf


In [9]:
# Example Usecase
from pynq import MMIO
from pynq import Overlay
ol = Overlay("uartlitetest.bit")
ol.download()

ADDRESS = 0x42c00000  # Address of the ip core

MMIO Debug: MMIO(address, size) = (42c00000, 10000 bytes).


In [11]:
# Setup axi core
uart = MMIO(ADDRESS,0x10000, debug=False)
setup_ctrl_reg(uart)
# Check Status
print(current_status(uart))

# Loopback test
write_uart(uart,'heldsklfjs\n')
print(current_status(uart))
#print(readline_uart(uart))


{'RX_VALID': 0, 'RX_FULL': 0, 'TX_EMPTY': 1, 'TX_FULL': 0, 'IS_INTR': 0, 'OVERRUN_ERR': 0, 'FRAME_ERR': 0, 'PARITY_ERR': 0}
{'RX_VALID': 1, 'RX_FULL': 0, 'TX_EMPTY': 0, 'TX_FULL': 0, 'IS_INTR': 0, 'OVERRUN_ERR': 0, 'FRAME_ERR': 0, 'PARITY_ERR': 0}


KeyboardInterrupt: 